In [6]:
# Step 1: Clone the repository from GitHub
import os
import shutil

repo_path = '/content/OmniParser'

# Change to /content first to avoid being inside the directory we want to remove
os.chdir('/content')

# Remove directory if it already exists
if os.path.exists(repo_path):
    print(f"🗑️ Removing existing directory: {repo_path}")
    shutil.rmtree(repo_path)
    print("✅ Directory removed")

print("📦 Cloning repository from GitHub...")
!git clone https://github.com/bilalsxadad1231231/ui-grounding-omniparser.git /content/OmniParser

# Change to the repository directory
os.chdir('/content/OmniParser')
print(f"✅ Repository cloned! Current directory: {os.getcwd()}")
%pip install -r requirements.txt

# List files to verify
print("\n📁 Files in repository:")
!ls -la


🗑️ Removing existing directory: /content/OmniParser
✅ Directory removed
📦 Cloning repository from GitHub...
Cloning into '/content/OmniParser'...
remote: Enumerating objects: 1062, done.
remote: Counting objects: 100% (1062/1062), done.
remote: Compressing objects: 100% (459/459), done.
remote: Total 1062 (delta 573), reused 1049 (delta 560), pack-reused 0 (from 0)
Receiving objects: 100% (1062/1062), 36.35 MiB | 23.29 MiB/s, done.
Resolving deltas: 100% (573/573), done.
✅ Repository cloned! Current directory: /content/OmniParser
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-proto 1.37.0 requires protobuf<7


📁 Files in repository:
total 236
drwxr-xr-x 9 root root   4096 Nov 18 18:13 .
drwxr-xr-x 1 root root   4096 Nov 18 18:13 ..
-rw-r--r-- 1 root root  13664 Nov 18 18:13 api_client.py
-rw-r--r-- 1 root root   9695 Nov 18 18:13 api_yolo_ocr.py
-rw-r--r-- 1 root root 106050 Nov 18 18:13 colab_test.ipynb
drwxr-xr-x 2 root root   4096 Nov 18 18:13 docs
drwxr-xr-x 2 root root   4096 Nov 18 18:13 eval
drwxr-xr-x 2 root root   4096 Nov 18 18:13 flash_attn
drwxr-xr-x 8 root root   4096 Nov 18 18:13 .git
-rw-r--r-- 1 root root    570 Nov 18 18:13 .gitignore
drwxr-xr-x 2 root root   4096 Nov 18 18:13 imgs
-rw-r--r-- 1 root root  19045 Nov 18 18:13 LICENSE
drwxr-xr-x 5 root root   4096 Nov 18 18:13 omnitool
-rw-r--r-- 1 root root   3697 Nov 18 18:13 README_API.md
-rw-r--r-- 1 root root   5530 Nov 18 18:13 README.md
-rw-r--r-- 1 root root    352 Nov 18 18:13 requirements.txt
-rw-r--r-- 1 root root   2697 Nov 18 18:13 SECURITY.md
-rw-r--r-- 1 root root  21672 Nov 18 18:13 streamlit_yolo_ocr.py
drwxr-

In [ ]:
# Step 2: Install required dependencies
print("📥 Installing dependencies...")
%pip install -q streamlit torch torchvision easyocr paddleocr pillow numpy opencv-python-headless ultralytics supervision transformers

print("✅ Dependencies installed!")


📥 Installing dependencies...
✅ Dependencies installed!


In [7]:
# Step 3: Download YOLO model weights
import os
from pathlib import Path

print("📥 Downloading YOLO model weights...")

# Install huggingface-cli if not available
%pip install -q huggingface_hub

# Create weights directory if it doesn't exist
weights_dir = Path('weights/icon_detect')
weights_dir.mkdir(parents=True, exist_ok=True)

# Download model from HuggingFace (OmniParser v2)
print("Downloading icon_detect model...")
!huggingface-cli download microsoft/OmniParser-v2.0 --local-dir weights --repo-type model --include "icon_detect/*"

# Verify model file exists
model_path = weights_dir / 'model.pt'
if model_path.exists():
    print(f"✅ Model downloaded successfully: {model_path}")
    print(f"   Size: {model_path.stat().st_size / (1024*1024):.2f} MB")
else:
    print("⚠️ Model file not found. Trying alternative download method...")
    # Alternative: Direct download if huggingface-cli doesn't work
    import urllib.request
    model_url = "https://huggingface.co/microsoft/OmniParser-v2.0/resolve/main/icon_detect/model.pt"
    print(f"Downloading from: {model_url}")
    urllib.request.urlretrieve(model_url, str(model_path))
    if model_path.exists():
        print(f"✅ Model downloaded: {model_path}")
    else:
        print("❌ Failed to download model")


📥 Downloading YOLO model weights...
⚠️  Warning: 'huggingface-cli download' is deprecated. Use 'hf download' instead.
Fetching 4 files:   0% 0/4 [00:00<?, ?it/s]Downloading 'icon_detect/train_args.yaml' to 'weights/.cache/huggingface/download/icon_detect/yY0w53yW-cpD7ut1YupPAooN03k=.1324d38323acf956b9c10634adf607ba758bfccc.incomplete'

train_args.yaml: 1.69kB [00:00, 7.44MB/s]
Download complete. Moving file to weights/icon_detect/train_args.yaml

LICENSE: 34.5kB [00:00, 61.5MB/s]
Download complete. Moving file to weights/icon_detect/LICENSE
Fetching 4 files:  25% 1/4 [00:00<00:00,  5.42it/s]
icon_detect/model.pt:   0% 0.00/40.6M [00:00<?, ?B/s]Downloading 'icon_detect/model.yaml' to 'weights/.cache/huggingface/download/icon_detect/nN0-2nKCzw1Nohw9iAmfhixZSh8=.8372bb89bf54d620e1548382735675bf47241ee8.incomplete'


model.yaml: 1.14kB [00:00, 7.56MB/s]A
Download complete. Moving file to weights/icon_detect/model.yaml

icon_detect/model.pt: 100% 40.6M/40.6M [00:00<00:00, 54.2MB/s]
Download

In [ ]:
# Step 4: Verify files are in place
import os

print("🔍 Verifying setup...")
print(f"Current directory: {os.getcwd()}")

# Check for streamlit file
if os.path.exists('streamlit_yolo_ocr.py'):
    print("✅ streamlit_yolo_ocr.py found")
else:
    print("❌ streamlit_yolo_ocr.py not found")
    print("Available Python files:")
    for f in os.listdir('.'):
        if f.endswith('.py'):
            print(f"  - {f}")

# Check for model
if os.path.exists('weights/icon_detect/model.pt'):
    print("✅ YOLO model found")
else:
    print("⚠️ YOLO model not found - you may need to download it manually")


🔍 Verifying setup...
Current directory: /content/OmniParser
✅ streamlit_yolo_ocr.py found
✅ YOLO model found


In [4]:
%pip install pyngrok

In [9]:
# Step 5: Run Streamlit app
import subprocess
import threading
from pyngrok import ngrok
import time
import os

print("🚀 Starting Streamlit app...")

# Install pyngrok for public URL
%pip install -q pyngrok

# Kill any existing ngrok processes and tunnels
print("🔍 Checking for existing ngrok processes...")
try:
    # Disconnect all existing ngrok tunnels using pyngrok API
    try:
        ngrok.kill()  # Kill all ngrok tunnels
        print("✅ Disconnected existing ngrok tunnels")
    except:
        pass
    
    # Also kill ngrok processes at system level
    os.system("pkill -f ngrok 2>/dev/null")
    time.sleep(2)  # Wait for processes to terminate
    print("✅ Killed existing ngrok processes")
except Exception as e:
    print(f"⚠️ Could not kill ngrok processes: {e}")

# Also kill any existing streamlit processes on port 8501
try:
    os.system("fuser -k 8501/tcp 2>/dev/null || lsof -ti:8501 | xargs kill -9 2>/dev/null")
    time.sleep(1)
    print("✅ Freed port 8501")
except:
    pass

# Configure ngrok with your authtoken
NGROK_AUTHTOKEN = "2E1YdKiDrWlzgs0dGVoDQ9Tns41_4yEpFhz67Ym3Mc966adNV"
ngrok.set_auth_token(NGROK_AUTHTOKEN)
print("✅ Ngrok authtoken configured")

# Function to run streamlit
def run_streamlit():
    subprocess.run([
        'streamlit', 'run', 'streamlit_yolo_ocr.py',
        '--server.port', '8501',
        '--server.address', '0.0.0.0',
        '--server.headless', 'true'
    ])

# Start streamlit in background thread
thread = threading.Thread(target=run_streamlit, daemon=True)
thread.start()

# Wait for streamlit to start
print("⏳ Waiting for Streamlit to start...")
time.sleep(5)

# Create public URL with ngrok
try:
    public_url = ngrok.connect(8501)
    print(f"\n🌐 Public URL: {public_url}")
    print(f"\n📱 Open this URL in your browser to access the app!")
    print(f"\n💡 Local URL: http://localhost:8501")
except Exception as e:
    print(f"⚠️ Could not create ngrok tunnel: {e}")
    print(f"💡 Try accessing: http://localhost:8501")


🚀 Starting Streamlit app...
🔍 Checking for existing ngrok processes...
✅ Disconnected existing ngrok tunnels
✅ Killed existing ngrok processes
✅ Freed port 8501
✅ Ngrok authtoken configured
⏳ Waiting for Streamlit to start...

🌐 Public URL: NgrokTunnel: "https://7d75e6ea4f5b.ngrok-free.app" -> "http://localhost:8501"

📱 Open this URL in your browser to access the app!

💡 Local URL: http://localhost:8501


# FastAPI Server for YOLO OCR Detection

This section runs the FastAPI server that accepts images and returns bounding boxes.


In [ ]:
# Step 6: Install FastAPI dependencies
print("📥 Installing FastAPI dependencies...")
%pip install -q fastapi uvicorn[standard] python-multipart requests

print("✅ FastAPI dependencies installed!")


In [10]:
# Step 7: Run FastAPI server
import subprocess
import threading
from pyngrok import ngrok
import time
import os

print("🚀 Starting FastAPI server...")

# Kill any existing ngrok processes and tunnels
print("🔍 Checking for existing ngrok processes...")
try:
    # Disconnect all existing ngrok tunnels using pyngrok API
    try:
        ngrok.kill()  # Kill all ngrok tunnels
        print("✅ Disconnected existing ngrok tunnels")
    except:
        pass
    
    # Also kill ngrok processes at system level
    os.system("pkill -f ngrok 2>/dev/null")
    time.sleep(2)  # Wait for processes to terminate
    print("✅ Killed existing ngrok processes")
except Exception as e:
    print(f"⚠️ Could not kill ngrok processes: {e}")

# Kill any existing processes on port 8000
print("🔍 Checking for existing processes on port 8000...")
try:
    os.system("fuser -k 8000/tcp 2>/dev/null || lsof -ti:8000 | xargs kill -9 2>/dev/null")
    time.sleep(1)
    print("✅ Freed port 8000")
except:
    pass

# Verify api_yolo_ocr.py exists
if os.path.exists('api_yolo_ocr.py'):
    print("✅ api_yolo_ocr.py found")
else:
    print("❌ api_yolo_ocr.py not found")
    print("Available Python files:")
    for f in os.listdir('.'):
        if f.endswith('.py'):
            print(f"  - {f}")

# Function to run FastAPI server
def run_fastapi():
    import uvicorn
    uvicorn.run("api_yolo_ocr:app", host="0.0.0.0", port=8000, log_level="info")

# Start FastAPI in background thread
api_thread = threading.Thread(target=run_fastapi, daemon=True)
api_thread.start()

# Wait for FastAPI to start
print("⏳ Waiting for FastAPI server to start...")
time.sleep(8)

# Create public URL with ngrok
try:
    api_public_url = ngrok.connect(8000)
    print(f"\n🌐 API Public URL: {api_public_url}")
    print(f"💡 Local API URL: http://localhost:8000")
    print(f"\n📚 API Documentation:")
    print(f"   - Swagger UI: {api_public_url}/docs")
    print(f"   - ReDoc: {api_public_url}/redoc")
    print(f"   - Health Check: {api_public_url}/health")
    print(f"   - Detect Endpoint: {api_public_url}/detect")
except Exception as e:
    print(f"⚠️ Could not create ngrok tunnel: {e}")
    print(f"💡 Try accessing: http://localhost:8000")


🚀 Starting FastAPI server...
🔍 Checking for existing ngrok processes...
✅ Disconnected existing ngrok tunnels
✅ Killed existing ngrok processes
🔍 Checking for existing processes on port 8000...
✅ Freed port 8000
✅ api_yolo_ocr.py found
⏳ Waiting for FastAPI server to start...


Exception in thread Thread-7 (run_fastapi):
Traceback (most recent call last):
  File "/usr/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipython-input-590810372.py", line 49, in run_fastapi
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/main.py", line 593, in run
    server.run()
  File "/usr/local/lib/python3.12/dist-packages/uvicorn/server.py", line 67, in run
    return asyncio_run(self.serve(sockets=sockets), loop_factory=self.config.get_loop_factory())
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/runners.py", line 195, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


🌐 API Public URL: NgrokTunnel: "https://63c419f3e949.ngrok-free.app" -> "http://localhost:8000"
💡 Local API URL: http://localhost:8000

📚 API Documentation:
   - Swagger UI: NgrokTunnel: "https://63c419f3e949.ngrok-free.app" -> "http://localhost:8000"/docs
   - ReDoc: NgrokTunnel: "https://63c419f3e949.ngrok-free.app" -> "http://localhost:8000"/redoc
   - Health Check: NgrokTunnel: "https://63c419f3e949.ngrok-free.app" -> "http://localhost:8000"/health
   - Detect Endpoint: NgrokTunnel: "https://63c419f3e949.ngrok-free.app" -> "http://localhost:8000"/detect
